In [1]:
# Speed comparison SH
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdLog_SH_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201228/logs_20201228_zt_88_03_day_88data.tar.gz/mdLog_SH_***'))
assert(len(path1) == 1)
print(path1[0])

logSH = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
logSH1 = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
fasterPerc = []
fasterTm25 = []
fasterTm = []
fasterTm75 = []
slowerTm25 = []
slowerTm = []
slowerTm75 = []

display(logSH['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSH1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdLog_SH_20201228_0923.csv
/mnt/data/20201228/logs_20201228_zt_88_03_day_88data.tar.gz/mdLog_SH_20201228_0835.csv
original data source type: 
[22 23]
new data source type: 
[29]
['08:35:46.240' '08:35:47.060' '08:35:47.100' ... '15:12:42.000'
 '15:12:43.000' '15:12:43.590']
['09:23:56.000' '09:23:57.000' '09:23:58.000' ... '15:39:26.000'
 '15:39:26.280' '15:39:29.000']


array([1, 2, 5, 6, 7])

array([1])

In [15]:
logSH1[logSH1['StockID'].isin(in_dex)]['time'].min()

120739000

In [3]:
### set here ***
columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]

num = 1
### set here ***
    
if num == 2:
    for i in range(0, num):
        in_dex = [16, 300, 852, 905]
        data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & \
                      (logSH['source'] == np.sort(logSH['source'].unique())[i])]
        data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == np.sort(logSH1['source'].unique())[i])]
        data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
        data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
        n1 = len(data1_1["StockID"].unique())
        n2 = len(data2_1["StockID"].unique())

        print(n1)
        print(n2)
        if n1 != n2:
            sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
            data1_1 = data1_1[data1_1["StockID"].isin(sl)]
            data2_1 = data2_1[data2_1["StockID"].isin(sl)]
        for cols in ['cum_amount']:
            data1_1[cols] = data1_1[cols].round(2)
            data2_1[cols] = data2_1[cols].round(2)
        data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
        n1 = test["sequenceNo_x"].count()
        n2 = test["sequenceNo_y"].count()
        len1 = len(test)
        print(n1)
        print(n2)
        print(len1)
        print("-----------------------------------------------")
        if n2 < len1:
            display("test is not complete:")
            display(test[np.isnan(test["sequenceNo_y"])])
            print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        if (len1 == n2) & (n1 < len1):
            display("baseline is not complete:")
            display(test[np.isnan(test["sequenceNo_x"])])
            display(n2-n1)
            print((n2-n1)/n1)

else:
    in_dex = [16, 300, 852, 905]
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & \
                  (logSH['source'] == logSH['source'].max())]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == logSH1['source'].max())]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())

    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(2)
        data2_1[cols] = data2_1[cols].round(2)
    data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)  
    

in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

display(data1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(data2['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())


### set here ***
columns = ["StockID", "cum_volume", "close", "openPrice"]
### set here ***

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])


1768
1455
5053385
5053383
5053386
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
337702,1294473.0,1.609120e+15,21588999.0,23.0,600864,SH,94241000.0,72487167,6.608833e+08,9.44,...,6400,9.00,29959,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4947753,13665654.0,1.609138e+15,140667834.0,23.0,600085,SH,145444000.0,9475812,2.236906e+08,23.46,...,800,23.59,11882,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4979936,13747279.0,1.609139e+15,141587960.0,23.0,600733,SH,145604000.0,69869439,6.084579e+08,8.62,...,28200,8.90,36654,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5.936614764163031e-07


array([1, 5])

array([1])

11411
11411
11411


In [16]:
pd.set_option('max_columns', 200)
columns = ["StockID", "time", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid1q", "bid2q",
           "ask1p", "ask2p", "ask1q", "ask2q","openPrice", "numTrades"]
logSH[(logSH['StockID'] == 600085) & (logSH['time'] >= 145441000) & (logSH['source'] == 23)][columns].head(20)

,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q,openPrice,numTrades
13662596,600085,145441000,9475812,2.236906e+08,23.46,23.46,23.45,34900,22300,23.47,23.48,2000,8600,23.59,11882
13665654,600085,145444000,9475812,2.236906e+08,23.46,23.46,23.45,34900,22500,23.47,23.48,2000,8600,23.59,11882
13665655,600085,145444000,9476012,2.236953e+08,23.46,23.46,23.45,34700,22500,23.47,23.48,2000,8600,23.59,11883
13668635,600085,145447000,9476212,2.237000e+08,23.46,23.46,23.45,34500,22600,23.47,23.48,3700,8600,23.59,11884
13671584,600085,145450000,9489212,2.240050e+08,23.46,23.46,23.45,25900,22500,23.47,23.48,500,8600,23.59,11889
13674563,600085,145453000,9499612,2.242490e+08,23.46,23.47,23.46,100,33400,23.48,23.49,8600,1500,23.59,11896
13677473,600085,145456000,9499812,2.242537e+08,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897
13680605,600085,145459000,9499812,2.242537e+08,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897
13683786,600085,145502000,9499812,2.242537e+08,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897
13686956,600085,145505000,9500412,2.242678e+08,23.48,23.47,23.46,1500,35400,23.48,23.49,6700,1500,23.59,11900


In [14]:
logSH1[(logSH1['StockID'] == 600085) & (logSH1['time'] >= 145441000)][columns].head(20)

,StockID,time,cum_volume,cum_amount,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q,openPrice,numTrades,time
5520329,600085,145441000,9475812,223690609.0,23.46,23.46,23.45,34900,22300,23.47,23.48,2000,8600,23.59,11882,145441000
5521602,600085,145444000,9476012,223695301.0,23.46,23.46,23.45,34700,22500,23.47,23.48,2000,8600,23.59,11883,145444000
5521603,600085,145444000,9476012,223695301.0,23.46,23.46,23.45,34700,22500,23.47,23.48,2000,8600,23.59,11883,145444000
5522821,600085,145447000,9476212,223699993.0,23.46,23.46,23.45,34500,22600,23.47,23.48,3700,8600,23.59,11884,145447000
5524054,600085,145450000,9489212,224005007.0,23.46,23.46,23.45,25900,22500,23.47,23.48,500,8600,23.59,11889,145450000
5525280,600085,145453000,9499612,224249003.0,23.46,23.47,23.46,100,33400,23.48,23.49,8600,1500,23.59,11896,145453000
5526479,600085,145456000,9499812,224253699.0,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897,145456000
5527814,600085,145459000,9499812,224253699.0,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897,145459000
5529113,600085,145502000,9499812,224253699.0,23.48,23.47,23.46,100,33400,23.48,23.49,8400,1500,23.59,11897,145502000
5530426,600085,145505000,9500412,224267785.0,23.48,23.47,23.46,1500,35400,23.48,23.49,6700,1500,23.59,11900,145505000


In [17]:
logSH1[(logSH1['StockID'] == 600410) & (logSH1['cum_volume'] == 9535878)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
1050911,1608270080549974,17349761,29,600410,SH,134133000,9535878,88547322.0,9.17,9.17,...,9.2,9.21,9.22,8600,3020,13300,10200,15900,9.26,8679


In [18]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)
logSH1[(logSH1['StockID'] == 600410) & (logSH1['cum_volume'] >= 9535878)][['StockID', 'time', 'cum_volume', 'close', 'bid1p', 'bid2p', 'ask1p', 'ask2p', 'bid1q', 'bid2q', 'ask1q', 'ask2q', 'numTrades']].head()

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q,numTrades
1050911,600410,134133000,9535878,9.17,9.17,9.16,9.18,9.19,6500,38600,8600,3020,8679
1051889,600410,134136000,9539878,9.17,9.17,9.16,9.18,9.19,2500,38600,9100,3020,8684
1051890,600410,134136000,9539878,9.17,9.17,9.16,9.18,9.19,2500,38600,9100,3020,8684
1053021,600410,134139000,9542378,9.17,9.16,9.15,9.17,9.18,38600,28093,1200,9600,8689
1054848,600410,134144000,9542378,9.17,9.16,9.15,9.17,9.18,38600,28093,1200,10100,8689


In [20]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)
logSH[(logSH['StockID'] == 600410) & (logSH['cum_volume'] >= 9535878) & (logSH['source'] == 23)][['StockID', 'time', 'cum_volume', 'close', 'bid1p', 'bid2p', 'ask1p', 'ask2p', 'bid1q', 'bid2q', 'ask1q', 'ask2q', 'numTrades']].head()

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q,numTrades
9568647,600410,134133000,9535878,9.17,9.17,9.16,9.18,9.19,6500,38600,8600,3020,8679
9570956,600410,134136000,9535878,9.17,9.17,9.16,9.18,9.19,6500,38600,9100,3020,8679
9570957,600410,134136000,9539878,9.17,9.17,9.16,9.18,9.19,2500,38600,9100,3020,8684
9573382,600410,134139000,9542378,9.17,9.16,9.15,9.17,9.18,38600,28093,1200,9600,8689
9577172,600410,134144000,9542378,9.17,9.16,9.15,9.17,9.18,38600,28093,1200,10100,8689


In [17]:
import pandas as pd


path = np.array(glob.glob('/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdTradeLog_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201228/logs_20201228_zt_88_03_day_88data.tar.gz/mdTradeLog_***'))
assert(len(path1) == 1)
print(path1[0])

SH = pd.read_csv(path1[0], encoding="utf-8", index_col=False)
SH1 = pd.read_csv(path[0], encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
####
# SH = SH[SH['TransactTime'] >= SH1['TransactTime'].min()]
####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)


/mnt/data/20201228/zt_7401_20201228_10.235.16.80.tar.gz/Logs_full/mdTradeLog_20201228_0923.csv
/mnt/data/20201228/logs_20201228_zt_88_03_day_88data.tar.gz/mdTradeLog_20201228_0835.csv
1768
1453
{688128, 688129, 688133, 688135, 688138, 688139, 688155, 688156, 688157, 688158, 688159, 688160, 688165, 688166, 688168, 688169, 688177, 688178, 688179, 688180, 688181, 688185, 688186, 688188, 688189, 688196, 688198, 688199, 688200, 688202, 688208, 688215, 688218, 688219, 688221, 688222, 688228, 688229, 688233, 688256, 688258, 688266, 688268, 688277, 688278, 688286, 688288, 688289, 688298, 688299, 688300, 688301, 688308, 688309, 688310, 688311, 688312, 688313, 688318, 688321, 688330, 688333, 688335, 688336, 688338, 688339, 688356, 688357, 688358, 688360, 688363, 688365, 688366, 688368, 688369, 688377, 688378, 688379, 688386, 688388, 688389, 688390, 688393, 688396, 688398, 688399, 688408, 688418, 688466, 688488, 688500, 688505, 688508, 688513, 688516, 688518, 688519, 688520, 688521, 688526, 68852

In [6]:
SH1[SH1['SecurityID'] == 601336]['TransactTime'].min()

95228100